# Advanced Classification using Machine Learning in Health Care (Diagnosis of Covid using chest X-rays)

Project done by Uday Kumar S

Roll no. 20ETMC412015

## Introduction

X-rays are widely used in Medical Practice. They can be used to identify various diseases. However, a diagnosis depends on a doctor's experience, which can lead to improper treatment. Modern methods of Artificial Intelligence and pattern recognition make it possible to create expert systems that allow you to establish a diagnosis automatically.

There are two approaches we can take for the classification of images :

    1. Different classical methods and their comparison
    2. Convolutional Neural Networks

## Approach taken in this program

1. Download and preliminary transormation of images
2. Creating Image features
3. Comparing differnent classical classification methods
4. Building and fitting of Convolutional Neural Network

## Objectives

* Download and transform images.
* Create features of images.
* Build different classification models.
* Build CNN models.
* Build a diagnosis based on X-ray photos.

## Packages Required

* Python
* OS
* Numpy
* Glob
* SeaBorn
* Matplotlib
* Mahotas
* Keras
* Scikit-learn
* Pandas

### Required libraries installation commands

In [1]:
#pip install numpy

In [2]:
#pip install matplotlib

In [3]:
#pip install glob2

In [4]:
#pip install seaborn

In [4]:
#pip install mahotas

In [6]:
#pip install os-sys

In [7]:
#pip install keras

In [8]:
#pip install skillsnetwork

In [9]:
#conda install scikit-learn

In [10]:
#conda install -c conda-forge tensorflow --yes

In [11]:
#pip install tensorflow-macos

In [12]:
#conda install -c apple tensorflow-deps

In [13]:
#pip install tensorflow-metal

In [14]:
#pip install tensorflow_datasets

In [ ]:
#conda install -c conda-forge keras --yes

### Importing the required libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mahotas as mh
import seaborn as sns
from glob import glob
import tensorflow as tf

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
#Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/Users/rogudays/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/__init__.py)

## Importing the Data-Set

In [4]:
import skillsnetwork
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-machine-learning-classification/LargeData/Covid19-dataset.zip", overwrite=True)
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-machine-learning-classification/NN.zip", overwrite=True)

  0%|          | 0/672 [00:00<?, ?it/s]

Saved to '.'


  0%|          | 0/4 [00:00<?, ?it/s]

Saved to '.'


Images have to be of the same size for classification. In order to achieve this, let's create a global variable that will determine the size (height and width) for image resizing. Both are 224 in our case.

In [ ]:
IMM_SIZE=244

For convenience, we create a function that downloads and displays all the pictures from a specified directory. In order to classify images, all pictures should be placed in sub-directories. The names of these subdirectories are actually class names. In our case, the images are X-rays which should be placed into subfolders with the names of diagnose. First of all, we should create a list of subfolders, which is a list of possible disease classes. In our case, 
it will be:
* Normal
* COVID
* Pneumonia

Next we need to create a list of all images. After that, let's download and preprocess all the images:

1. Download by mahotas.imread()
2. It is necessary to resize the images to (IMM_SIZE x IMM_SIZE). If an image is gray-colored it is presented as a 2D matrix: (height, width).jpg and png images are 3D (height, width, 3 or 4). To do this, we should use: mahotas.resize_to()
3. If the third parameter of an image shape equals 4, it means alpha channel. We can remove it using slice image[:,:,:3].
4. Then we need to transform all the images to gray 2D format by: mahotas.colors.rgb2grey()

The function returns an array of tuples[image, class name].

In [ ]:
def get_data(folder):
    class_names=[f for f in os.listdir(folder) if not f.startswith('.')] # create a list of subfolders
    data=[]
    # class_names = ['Covid','Normal','Viral Pneumonia']
    print(class_names)
    for t, f in enumerate(class_names):
        images = glob(folder + "/" + f + "/*") # Create a list of files
        print('Downloading: ',f)
        fig=plt.figure(figsize=(50,50))
        for im_n, im in enumerate(images):
            plt.gray() # set gey colormap of images
            image=mh.imread(im)
            if len(image.shape)>2:
                image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE, image.shape[2]]) # resize of RGB and png images
            else:
                image = mh.resize_to(image, [IMM_SIZE, IMM_SIZE]) # resize of grey images
            if len(image.shape)>2:
                image = mh.colors.rgb2grey(image[:,:,:3], dtype=np.uint8) #change of colormap of images alpha channel delete
            plt.subplot(int(len(images)/5)+1,5,im_n+1) # Create a table of images
            plt.imshow(image)
            data.append([image, f])
        plt.show()
    return np.array(data)

For training and testing, all the pictures should be divided into training and test groups and located in separate folders. Let's upload all the images to the corresponding arrays.

In [ ]:
d='/Users/rogudays/Downloads/Covid19-dataset/train'
train=get_data(d)

d='/Users/rogudays/Downloads/Covid19-dataset/test'
val=get_data(d)

In [ ]:
print('Train Shape', train.shape) # Size of the training DataSet
print('Test Shape', val.shape) # Size of the test DataSet
print('Image Size', train[0][0].shape) # Size of image

As we can see, the training DataSet consists of 251 images and the test one consists of 66 images. All the images are in grey 2D (244x244) format.

## Data Visualization

Let's visualize our data and see what exactly are we working with. We use Seaborn to plot the number of images in all the classes. You can see what the output looks like.

In [ ]:
l=[]
for i in train:
    l.append(i[1])
sns.set_style('darkgrid')
sns.countplot(l)

let's also visualize the first image from the Viral Pneumonia, Normal and Covid classes in the training DataSet:

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(train[np.where(train[:,1]=='Viral Pneumonia')[0][0]][0])
plt.title('Viral Pneumonia')

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(train[np.where(train[:,1]=='Covid')[0][0]][0])
plt.title('Covid')

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(train[np.where(train[:,1]=='Normal')[0][0]][0])
plt.title('Normal')

## Image Features Creation

To classify objects, you need to transform the data set so that the input is a set of features and the output is an object class. An image is a matrix of pixels. Each pixel is a color. Therefore, it is impossible to submit a bare picture into a classical classifier's input. It is necessary to turn each picture into a set of certain features.

Mahotas makes it easy to calculate features of an image. The corresponding functions are found in the (mahotas.features) submodule. The Haralick set of texture features are based on textures, that is, they distinguish between structured and unstructured images, as well as various repetitive structures. With the help of Mahotas, these features are calculated very easily: haralick_features = mh.features.haralick(image) haralick_features_mean=np.mean(haralick_features, axis=0) haralick_features_all=np.ravel(haralick_features) The mh.features.haralick function returns a 4 x 13 array which should be transformed into 1D by NumPy.ravel(). The first dimension is the four possible directions in which the features are calculated (vertical, horizontal, and two diagonals). If we are not interested in any particular direction, then we can average the features in all directions. Alternatively, you can use all the characteristics individually(variable haralick_features_all). The choice depends on the properties of a particular dataset. We decided that the vertical and horizontal featues should be stored separately in our case, so we use haralick_features_all.

We should make a function for the DataSet features creation.

In [ ]:
def create_features(data):
    features=[]
    labels=[]
    for image, label in data:
        features.append(mh.features.haralick(image).ravel())
        labels.append(label)
    features=np.array(features)
    labels=np.array(labels)
    return(features, labels)

In [ ]:
features_train, labels_train = create_features(train)
features_test, labels_test = create_features(val)

## Comparing different classical classification methods

If we want to compare some classificators, we should use a Pipeline. A pipeline helps to chain multiple estimators into a single one. This is useful as there is often a fixed number of steps in data processing, for example, feature selection, normaliztion and classification. A pipeline serves multiple purposes here:

You only have to call fit() once to evaluate a whole sequence of estimators. 

You can grid search over parameters of all estimators in the pipeline at once.

Pipelines help to avoid leaking statistics from your test data into the trained model in cross-validation by ensuring that the same samples are used to train the transformers and predictors. All estimators in a pipeline, except the last one, should be transformers (i.e. should have a transform method). The last estimator may be of any type (transformer, classifier, etc).

The sklearn.pipeline module implements utilities to build a composite estimator, as a chain of transforms and estimators.

In order to test how it works, we will use LogisticRegression.

In [ ]:
clf = Pipeline([('preproc', StandardScaler()), ('classifier', LogisticRegression())])
clf.fit(features_train, labels_train)
scores_train = clf.score(features_train, labels_train)
scores_test = clf.score(features_test, labels_test)
print('Training DataSet accuracy: {: .1%}'.format(scores_train), 'Test DataSet accuracy: {: .1%}'.format(scores_test))
plot_confusion_matrix(clf, features_test, labels_test)  
plt.show() 

As we can see, the results are not bad. Conflusion matrix shows us how many mistaken predictions we got. it allows us to check other classifiers and compare the results.

We will test:

* Logistic Regression
* Nearest Neighbours
* Linear SVM
* RBF SVM
* Gaussian Process
* Decision Tree
* Random Forest
* Multi-layer Perceptron classifier
* Ada Boost
* Naive Bayes
* Quadratic Discriminant Analysis

In [ ]:
names = ["Logistic Regression", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
scores_train = []
scores_test = []
for name, clf in zip(names, classifiers):
    print("Fitting:", name)
    clf = Pipeline([('preproc', StandardScaler()), ('classifier', clf)])
    clf.fit(features_train, labels_train)
    score_train = clf.score(features_train, labels_train)
    score_test = clf.score(features_test, labels_test)
    scores_train.append(score_train)
    scores_test.append(score_test)

Let's print the results as a table.

In [ ]:
res = pd.DataFrame(index=names)
res['scores_train']=scores_train
res['scores_test']=scores_test
res.columns=['Test', 'Train']
res.index.name='Classifier accuracy'
pd.options.display.float_format='{:,.2f}'.format
print(res)

You can see that the calculations are very fast and that Logistic Regression and Neural Network show the best result for the test DataSet.

Let's compare the results on a plot.

In [ ]:
x = np.arange(len(names)) # the label locations
width = 0.35 # the width of the bars

fig, ax =plt.subplots()
rects1 = ax.bar(x - width/2, scores_train, width, label = 'Train')
rects2 = ax.bar(x + width/2, scores_train, width, label = 'Test')

# Add some test for labels, title, and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy of classifiers')
ax.set_xticks(x)
plt.xticks(rotation = 90)
ax.set_xticklabels(names)
ax.legend()

fig.tight_layout()
plt.show()

## Building and fitting Convolutional Neural Network

### Required libraries import

We will use Keras library for creating and training our model.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

## Data preprocessing and data augmentation

What is different about Convolutional Neural Networks is that we can submit images directly to the input. However, these images also equire pre-processing.

In particular, it is necessary to normalize the pixel color, i.e. to normalize them from the range [0, 255) to [0,1).

You also need to change the dimension of the input images because of Keras framework.

Image classes should be of numeric type instead of string.

The code below makes the necessary transformations.

In [ ]:
x_train=[]
y_train=[]
x_val=[]
y_val=[]

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)
    
# Normalize the data
x_train = np.array(x_train)/255
x_val = np.array(x_val)/255

# Reshaping input images
x_train = x_train.reshape(-1, IMM_SIZE, IMM_SIZE, 1)
x_val = x_val.reshape(-1, IMM_SIZE, IMM_SIZE, 1)

# Creating a dictionary of clases
lab = {}
for i, l in enumerate(set(y_train)):
    lab[l] = i
    
y_train = np.array([lab[l] for l in y_train])
y_val = np.array([lab[l] for l in y_val])

In [ ]:
print('Shape of the input DataSet :', x_train.shape)
print('Shape of the output DataSet :', y_train.shape)
print('Dictionary of classes :', lab)

## Data augmentation on the training data

We should perform data augmentation to fit our model better.

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.2,  # Randomly zoom image
    width_shift_range = 0.1,  # randomly shifts images horizontally (fraction of total_width)
    height_shift_range = 0.1,  # randomly shifts images vertically (fraction of total-height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False  # randomly flip images
)
datagen.fit(x_train)

## Model defining

Let's define a simple CNN model with 3 Convolutional layers followed by max-pooling layers. A dropout layer is added after the 3rd maxpool operation to avoid overfitting.

In [ ]:
model = sequential()
model.add(Conv2D(32, 1, padding='same', activation='relu', input_shape=(IMM_SIZE, IMM_SIZE, 1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 1, padding='same', activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, 1, padding='same', activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

In [ ]:
opt = Adam(lr=0.000001)
model.compile(optimizer = opt, loss = tf.keras.losses.SparceCategoricalCrossentropy(from_logits = True), metrics=['accuracy'])

Now, let's train our model for 2000 epochs. Admittedly, the fitting process is very slow. Therefore, we saved the fitted model to a file. To save time, we will upload the fitted model. If you would like, you can change the parameter fitting to True in our order to refit the model.

In [ ]:
fitting = False
fitting_save = False
epochs = 2

import pickle

if fitting:
    history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val), shuffle=True)
    if fitting_save:
        # serialize model to JSON
        model_json = model.to_json()
        with open('model.json','w') as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        mode.save_weights('model.h5')
        print('Save modelm to disk')
        with open('history.pickle','wb') as f:
            pickle.dump(history.history, f)
        with open('lab.pickle','wb') as f:
            pickle.dump(lab, f)
# load model
from keras.models import model_from_json
json_file = open('model.json','r')
loaded_model_json=json_file.read()
json_file.close()
model=model_from_json(loaded_model_json)
# load weights into a new model
model.load_weights('model.h5')
with open('history.pickle','rb') as f:
    history=pickle.load(f)
print('Loaded model from disk')

## Results evaluation

We will plot out training and validation accuracy along with the training and validation loss.

In [ ]:
acc=history['accuracy']
val_acc=history['val_accuracy']
loss=history['loss']
val_loss=history['val_loss']

epochs_range=range(epochs)

plt.figure(figsize=(15,15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')
plt.show()

We can see the accuracy and training and validation sets is the same. The loss function of validation and training sets is stable. It means that our CNN is fitted well and can be used for classification.

We can print out the classification report to see the precision and accuracy using model.predict_classes() and classification_report().

Also we can create a confusion matrix. Unfortunetly keras framework does not have plot_confusion_matrix() function. Therefore, we have to create it using pandas and Seaborn.heatmap()

In [ ]:
# Classification report
predictions= model.predict_classes(x_val)
predictions= predictions.reshape(1,-1)[0]
print(classification_report(y_val, predictions, target_names=lab.keys()))

# Confusion matrix
cm=pd.DataFrame(confusion_matrix(y_val, predictions, target_names=lab.keys()))
cm.index=['Predicted ' + s for s in lab.keys()]
cm.comlumns=['True' + s for s in lab.keys()] 
print(cm)

sns.heatmap(confusion_matrix(y_val, predictions), annot=True,
           xticklabels=list(lab.keys()), yticklabels=list(lab.keys()))
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
# Accuracy
z=model.predict_classes(x_train)==y_train
scores_train=sum(z+0)/len(z)
z=model.predict_classes(x_val)==y_val
score_test=sum(z+0)/len(z)
print('Training DataSet accuracy : {:.1%}'.format(scores_train), 'Test DataSet accuracy : {: .1%}'.format(score_test))